# Multi Document Search

In [1]:
import sys

sys.path.insert(0, '..')

import weaviate.classes as wvc
import weaviate
from collections import defaultdict
from app.schemas import ChunkWithScore, DocumentSearchResult
from weaviate.classes.query import Filter, GeoCoordinate, MetadataQuery, QueryReference, GroupBy


query = "l'île de Pâques"
client = weaviate.connect_to_local()
# print("SEARCH_MULTI_DOCUMENTS 0 ", query)
document_chunk = client.collections.get("DocumentChunk")
response = document_chunk.query.hybrid(
    query=query,
    return_metadata=wvc.query.MetadataQuery(score=True),
    query_properties=["text"],
    return_references=[QueryReference(link_on="belongsToDocument", return_properties=["document_id", "local_path", "original_public_path", "media_name", "max_score", "min_score"])]
)
# print("SEARCH_MULTI_DOCUMENTS 1 ", response)
client.close()

# Assume response is the result from the above query
grouped_results = defaultdict(list)
for chunk in response.objects:
    doc_ref = chunk.references['belongsToDocument'].objects[0].properties
    grouped_results[doc_ref['document_id']].append(chunk)

# Now `grouped_results` contains your data grouped by 'document_id'
# for document_id, chunks in grouped_results.items():
#     print(f"Media id: {document_id}, || Chunks: {chunks}")

documentsSearchResponse = []

for document_id, group in grouped_results.items():  # View by group
    # max_score = min(chunk for chunk in group)#group.max_score
    # min_score = min(chunk for chunk in group)#group.min_score
    currentDocumentChunksWithDistance = []
    for _documentChunk in group:
        score = _documentChunk.metadata.score
        cuurrentChunkWithScore = ChunkWithScore.model_validate({**_documentChunk.properties, "score": score})
        currentDocumentChunksWithDistance.append(cuurrentChunkWithScore)
    
    documentProperties = _documentChunk.references['belongsToDocument'].objects[0].properties
    
    documentSearchResult = DocumentSearchResult(
        document_id=documentProperties['document_id'],
        local_path=documentProperties['local_path'],
        original_public_path=documentProperties['original_public_path'],
        media_name=documentProperties['media_name'],
        min_score=min(chunk.score for chunk in currentDocumentChunksWithDistance),
        max_score=max(chunk.score for chunk in currentDocumentChunksWithDistance),
        chunks=currentDocumentChunksWithDistance
    )
    documentsSearchResponse.append(documentSearchResult)
for doc in documentsSearchResponse:
    print(doc.media_name)
    for chunk in doc.chunks:
        print(f"    -->  {chunk.score} | {chunk.text}")


Ile de Paques : la théorie de l’effondrement précolonial à nouveau démentie | Actu de science
    -->  1.0 | Une croyance encore bien ancrée dans les esprits laisse penser que jadis, les habitants de l'île de Pâques, de son nom autochtone Rapanoui, auraient surexploité leur environnement naturel, provoquant ainsi eux-mêmes leur disparition. Mais cette hypothèse a été maintes fois démentie et une nouvelle étude enfonce le clou.
    -->  0.7116864323616028 | En effet, l'île n'a jamais accueilli assez d'humains pour que cela ait entraîné leur Lorsque Rapa Nui est accosté par les colons européens, en 1722, il découvre d'imposantes statues de pierres érigées par milliers. Pourtant, il n'y a que 3 000 habitants sur place. Une question se pose alors, comment un si petit nombre d'individus a-t-il pu édifier de si nombreuses et si monumentales constructions ? Selon une idée popularisée par le géographe américain Jared Diamond il y a une vingtaine d'années, Rapa Nui était habité par de nombreux 

# Single Document Search

In [8]:
import sys

sys.path.insert(0, '..')

from weaviate.util import get_valid_uuid
import weaviate.classes as wvc
import weaviate
from collections import defaultdict
from app.schemas import ChunkWithScore, DocumentSearchResult
from weaviate.classes.query import Filter, GeoCoordinate, MetadataQuery, QueryReference, GroupBy


query = "l'île de Paques"
client = weaviate.connect_to_local()
# print("SEARCH_MULTI_DOCUMENTS 0 ", query)
document_chunk = client.collections.get("DocumentChunk")
response = document_chunk.query.hybrid(
    query=query,
    return_metadata=wvc.query.MetadataQuery(score=True),
    query_properties=["text"],
    filters=Filter.by_ref(link_on="belongsToDocument").by_property("document_id").equal(get_valid_uuid(uuid="9e80a4c3-7139-4f66-b450-e94cbbf06e2b")),
    return_references=[QueryReference(link_on="belongsToDocument", return_properties=["document_id", "local_path", "original_public_path", "media_name", "max_score", "min_score"])]
)
# print("SEARCH_MULTI_DOCUMENTS 1 ", response)
client.close()


# Assume response is the result from the above query
grouped_results = defaultdict(list)
for chunk in response.objects:
    doc_ref = chunk.references['belongsToDocument'].objects[0].properties
    grouped_results[doc_ref['document_id']].append(chunk)

# Now `grouped_results` contains your data grouped by 'document_id'
# for document_id, chunks in grouped_results.items():
#     print(f"Media id: {document_id}, || Chunks: {chunks}")

documentsSearchResponse = []

for document_id, group in grouped_results.items():  # View by group
    # max_score = min(chunk for chunk in group)#group.max_score
    # min_score = min(chunk for chunk in group)#group.min_score
    currentDocumentChunksWithDistance = []
    for _documentChunk in group:
        score = _documentChunk.metadata.score
        cuurrentChunkWithScore = ChunkWithScore.model_validate({**_documentChunk.properties, "score": score})
        currentDocumentChunksWithDistance.append(cuurrentChunkWithScore)
    
    documentProperties = _documentChunk.references['belongsToDocument'].objects[0].properties
    
    documentSearchResult = DocumentSearchResult(
        document_id=documentProperties['document_id'],
        local_path=documentProperties['local_path'],
        original_public_path=documentProperties['original_public_path'],
        media_name=documentProperties['media_name'],
        min_score=min(chunk.score for chunk in currentDocumentChunksWithDistance),
        max_score=max(chunk.score for chunk in currentDocumentChunksWithDistance),
        chunks=currentDocumentChunksWithDistance
    )
    documentsSearchResponse.append(documentSearchResult)
for doc in documentsSearchResponse:
    print(doc.media_name)
    for chunk in doc.chunks:
        print(f"    -->  {chunk.score} | {chunk.text}")


Ile de Paques : la théorie de l’effondrement précolonial à nouveau démentie | Actu de science
    -->  1.0 | Une croyance encore bien ancrée dans les esprits laisse penser que jadis, les habitants de l'île de Pâques, de son nom autochtone Rapanoui, auraient surexploité leur environnement naturel, provoquant ainsi eux-mêmes leur disparition. Mais cette hypothèse a été maintes fois démentie et une nouvelle étude enfonce le clou.
    -->  0.0 | En effet, l'île n'a jamais accueilli assez d'humains pour que cela ait entraîné leur Lorsque Rapa Nui est accosté par les colons européens, en 1722, il découvre d'imposantes statues de pierres érigées par milliers. Pourtant, il n'y a que 3 000 habitants sur place. Une question se pose alors, comment un si petit nombre d'individus a-t-il pu édifier de si nombreuses et si monumentales constructions ? Selon une idée popularisée par le géographe américain Jared Diamond il y a une vingtaine d'années, Rapa Nui était habité par de nombreux autochtones à m